In [1]:
%reset -f

import sys
import mne
import matplotlib.pyplot as plt
import numpy as np
import os.path as op
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from pathlib import Path, PurePath

pick_ch=['P9','P7','P5','P3','PO7','PO3','O1','P10','P8','P6','P4','PO8','PO4','O2','POz','Pz']

%run EEG_auxiliary_module.ipynb
%run general_tools.ipynb
mne.__version__
mne.set_log_level("WARNING")

infolder, outfolder = find_folder()
#outfolder = 'D:\\Ilya_study\\Analysis'
subs = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
subs = ['03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18']

# Preprocessing

### Running imagery preprocessing on all files

In [ ]:
%run EEG_auxiliary_module.ipynb
%run general_tools.ipynb
subs = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18] #1 is absent, 2 has to be processed separately 
for i,sub in enumerate(subs):
    update_progress(i/len(subs))
    preprocess_all_imag(sub=sub, filt=(0.1,50), resample=256, im_times=(-0.1,5.6), 
                        dims=3, trials = 8, blocks = 12)
    

### Running perception preprocessing on all files

In [2]:
%run EEG_auxiliary_module.ipynb
%run general_tools.ipynb
subs = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18] #1 is absent, 2 has to be processed separately 
for i,sub in enumerate(subs):
    update_progress(i/len(subs))
    preprocess_all_perc(sub=sub, filt=(0.1,50), resample=256, im_times=(-0.1,1), 
                        dims=[1,2], trials = 4, blocks = 12)

Progress: [###################-] 93.8%
(4800, 64, 564)
Overwriting existing file.
Starting averaging
No zscoring performed
Overwriting existing file.


# Computing discrimination

### Computing svm discrimination accuracy for perception

In [3]:
%run general_tools.ipynb
%run EEG_auxiliary_module.ipynb
%run make_features.ipynb
import numpy as np
subs = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
prc_time = (0.05,0.65)
data=list()
labels = list()
results=[]# Loading first S02 session
durs=[]
dictr = {"svm_prc_accuracy.pkl":results, 
        "svm_prc_duration.pkl":durs}
for i,j in enumerate(subs):
    update_progress(i/len(subs))
    fname = PurePath(outfolder,'S'+str(j).zfill(2)+'_perc_aver-epo.fif')
    epochs = mne.read_epochs(fname)
    cv=len(epochs['101'])
    X, Y = convert_epochs_to_2D_array(epochs, times = prc_time, pick_ch=pick_ch)
    #X,Y = epochs_to_stft_features(outfolder, fname, step = 76, times = prc_time, pick_ch = pick_ch)
    #X,Y = epochs_to_morlet_features(fname,times = prc_time,pick_ch = pick_ch)
    confusion, duration = run_eeg_svm(X, Y, cv)
    results.append(confusion)
    durs.append(duration)
    save_all_pickle(outfolder,dictr = dictr)
    fname = op.join(outfolder,'conf_mat_perc_'+str(j).zfill(2)+'.csv')
    np.savetxt(fname, confusion, delimiter=',')


Progress: [#-------------------] 5.9%
No PCA
No fft


KeyboardInterrupt: 

### Computing svm discrimination accuracy for imagery

In [2]:
%run general_tools.ipynb
%run EEG_auxiliary_module.ipynb
outfolder = 'D:\\Ilya_study\\Analysis'
subs = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
times = [(0.6,1.6),(1.6,2.6),(2.6,3.6),(3.6,4.6),(4.6,5.6),(0.6,5.6)]
data=list()
labels = list()
results=[]
durs=[]
dictr = {op.join(outfolder,"svm_img_accuracy.pkl"):results, op.join(outfolder,"svm_img_duration.pkl"):durs}
for k,time in enumerate(times):
    res = []
    dur = []
    for i,j in enumerate(subs):
        update_progress((k*len(subs)+i)/(len(subs)*len(times)))
        fname = op.join(outfolder,'S'+str(j).zfill(2)+'_imag_aver-epo.fif')
        epochs = mne.read_epochs(fname)
        cv=len(epochs['31'])
        X, Y = convert_epochs_to_2D_array(epochs, times = time)
        confusion, duration = run_eeg_svm(X, Y, cv)
        res.append(confusion)
        dur.append(duration)  
    results.append(res)
    durs.append(dur)
    save_all_pickle(dictr = dictr)
    #fname = op.join(outfolder,'conf_mat_imag_'+str(j).zfill(2)+'.csv')
    #np.savetxt(fname, confusion, delimiter=',')
os.system('say "your program has finished"')

Progress: [####################] 99.0%
No PCA
No fft
The overall accuracy is 59.6 and the duration is 0.2 minutes


1

# Preparing summary table

### Perception discrimination

In [5]:
# loading pickle file
import pickle
import os.path as op
from scipy.spatial.distance import squareform
import numpy as np
import pandas as pd
%run general_tools.ipynb
infolder, outfolder = find_folder()
outfolder = 'D:\\Ilya_study\\Analysis'
subs = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
%run EEG_auxiliary_module.ipynb
# fname = op.join(outfolder,'svm_prc_accuracy.pkl')
# infile = open(fname,'rb')
# data = pickle.load(infile)
# infile.close()
data = results
data = np.array(data)
lbls = ['up-unf','up-fam','inv-unf','inv-fam','up','inv','up-5','inv-5']
select_up = []
select_in = []
mat = []
for i in range(data.shape[0]):
    line = []
    line.append(np.mean(squareform(data[i,:25,:25])))
    line.append(np.mean(squareform(data[i,25:50,25:50])))
    line.append(np.mean(squareform(data[i,50:75,50:75])))
    line.append(np.mean(squareform(data[i,75:100,75:100])))
    line.append(np.mean(squareform(data[i,:50,:50])))
    line.append(np.mean(squareform(data[i,50:100,50:100])))
    select_up.append(squareform(isolate_perc_5(data[i,:50,:50])))
    select_in.append(squareform(isolate_perc_5(data[i,50:,50:])))
    line.append(np.mean(squareform(isolate_perc_5(data[i,:50,:50]))))
    line.append(np.mean(squareform(isolate_perc_5(data[i,50:,50:]))))
    mat.append(line)
mat = np.array(mat)
mat = pd.DataFrame(mat, columns=lbls)
select = pd.DataFrame(np.vstack((np.array(select_up),np.array(select_in))))
select = select.assign(kind = ['up']*len(subs)+['in']*len(subs), subs = subs*2) 
#select.to_csv(op.join(outfolder,'selected_perc.csv'))
mat.to_clipboard()
mat.mean()

up-unf     0.571499
up-fam     0.546133
inv-unf    0.597040
inv-fam    0.568196
up         0.560729
inv        0.583559
up-5       0.555348
inv-5      0.578186
dtype: float64

### Imagery discrimination

In [61]:
# loading pickle file
from scipy.spatial.distance import squareform
import numpy as np
import pandas as pd
from scipy.spatial.distance import squareform

labels = ['1','2','3','4','5','all']
mat = img_data.copy()
subs = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

def make_data_frame(labels,mat,subs):
    import pandas as  pd
    mat = mat.reshape((mat.shape[0]*mat.shape[1],mat.shape[2]))
    mat = pd.DataFrame(mat)
    mat = mat.assign(kind = [item for item in labels for i in range(data.shape[1])],
                    subs = subs*len(labels))
    return mat

%run general_tools.ipynb
infolder, outfolder = find_folder()

data = load_pkl(outfolder,'svm_img_accuracy.pkl')
data = np.array(data)
out = [];
for i in range(data.shape[0]):
    temp = []
    for j in range(data.shape[1]):
        temp.append(squareform(data[i,j,:,:]))
    out.append(temp)
out = np.array(out)
df_imag = make_data_frame(labels,out,subs)
df_imag.to_csv(outfolder+'\\selected_img.csv')
out = np.mean(out,2).transpose()
out = pd.DataFrame(out, columns=labels)
out.to_clipboard()

# Pairwise data analysis

### Preparing data frame with all the pairwise accuracies

In [53]:
# loading discrimination matrices
import os.path as op
%run general_tools.ipynb
infolder, outfolder = find_folder()
import pickle
import numpy as np
from scipy.spatial.distance import pdist, squareform
import pandas as pd

subs = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
fname = op.join(outfolder,'svm_prc_accuracy.pkl')
infile = open(fname,'rb')
data = pickle.load(infile)
infile.close()
data = np.array(data)
up = []
inv = []
for i in range(data.shape[0]):
    up.append(squareform(data[i,:50,:50]))
    inv.append(squareform(data[i,50:,50:]))
up = np.array(up).transpose()
inv = np.array(inv).transpose()
columns = ['up_S'+str(j).zfill(2) for j in subs]
columns = [*columns, *['inv_S'+str(j).zfill(2) for j in subs]]
disc = pd.DataFrame(np.hstack((up,inv)),columns = columns)

# adding the familiarity index
import itertools
%run EEG_auxiliary_module.ipynb
c = list(itertools.combinations(np.arange(50),2))
b = find_index_combs(np.arange(25),c)
d = find_index_combs(np.arange(25,50),c)    
vec = np.zeros(len(c)) 
vec[b] = 1
vec[d] = 2
disc = disc.assign(familiarity = vec)
disc["familiarity"].replace({0: "mixed", 1:'unf',2:'fam'}, inplace=True)

# adding theoretical observer
data = load_pkl('C:\\Users\\nemrodov\\Dropbox\\Ilya_study\\Stimuli','stims.pkl')
im_list = [x.flatten() for x in data]
im_list = np.array(im_list)
disc = disc.assign(TO = pdist(im_list, 'euclidean'))

# averaging upright and inverted excluding S16
subs = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18] # no S16
disc['upright'] = disc[['up_S'+str(j).zfill(2) for j in subs]].mean(1)
disc['inverted'] = disc[['inv_S'+str(j).zfill(2) for j in subs]].mean(1)

disc.head()

,up_S02,up_S03,up_S04,up_S05,up_S06,up_S07,up_S08,up_S09,up_S10,up_S11,...,inv_S11,inv_S12,inv_S13,inv_S14,inv_S15,inv_S16,inv_S17,inv_S18,familiarity,TO
0,0.318182,0.541667,0.500000,0.375000,0.416667,0.583333,0.541667,0.416667,0.458333,0.500000,...,0.541667,0.500000,0.416667,0.666667,0.416667,0.375000,0.708333,0.583333,unf,5041.518918
1,0.500000,0.333333,0.625000,0.583333,0.541667,0.500000,0.541667,0.375000,0.458333,0.583333,...,0.291667,0.416667,0.625000,0.541667,0.625000,0.541667,0.583333,0.541667,unf,5759.484439
2,0.318182,0.666667,0.666667,0.708333,0.375000,0.458333,0.500000,0.583333,0.541667,0.333333,...,0.333333,0.541667,0.500000,0.583333,0.458333,0.500000,0.583333,0.458333,unf,5575.778421
3,0.409091,0.458333,0.583333,0.583333,0.708333,0.458333,0.375000,0.583333,0.625000,0.708333,...,0.375000,0.458333,0.541667,0.583333,0.541667,0.541667,0.666667,0.708333,unf,4906.270580
4,0.454545,0.458333,0.375000,0.541667,0.458333,0.375000,0.541667,0.625000,0.416667,0.750000,...,0.416667,0.291667,0.541667,0.500000,0.500000,0.583333,0.500000,0.666667,unf,4949.899090


### Correlations 

#### Overall

In [88]:
%run general_tools.ipynb

print(disc[['upright','inverted','TO']].corr())
print(disc[['upright','inverted','TO']].corr(method=pearsonr_pval))

           upright  inverted        TO
upright   1.000000  0.440904 -0.013724
inverted  0.440904  1.000000  0.018451
TO       -0.013724  0.018451  1.000000
               upright      inverted        TO
upright   1.000000e+00  2.021173e-59  0.631324
inverted  2.021173e-59  1.000000e+00  0.518810
TO        6.313240e-01  5.188101e-01  1.000000


#### Separately for unfamiliar and famous

In [89]:
%run general_tools.ipynb

print(disc[disc.apply(lambda x: x['familiarity']!='mixed',axis=1)]
      .groupby('familiarity')[['upright','inverted','TO']].corr())

print(disc[disc.apply(lambda x: x['familiarity']!='mixed',axis=1)]
      .groupby('familiarity')[['upright','inverted','TO']].corr(method=pearsonr_pval))

                       upright  inverted        TO
familiarity                                       
fam         upright   1.000000  0.199819  0.144360
            inverted  0.199819  1.000000  0.019077
            TO        0.144360  0.019077  1.000000
unf         upright   1.000000  0.510832 -0.155017
            inverted  0.510832  1.000000  0.029028
            TO       -0.155017  0.029028  1.000000
                           upright      inverted        TO
familiarity                                               
fam         upright   1.000000e+00  4.982405e-04  0.012312
            inverted  4.982405e-04  1.000000e+00  0.742097
            TO        1.231225e-02  7.420965e-01  1.000000
unf         upright   1.000000e+00  2.424532e-21  0.007144
            inverted  2.424532e-21  1.000000e+00  0.616521
            TO        7.144122e-03  6.165211e-01  1.000000


#### Between imagery and perception

In [13]:
import pandas as pd
%run general_tools.ipynb
import os.path as op

infolder, outfolder = find_folder()
outfolder = 'D:\\Ilya_study\\Analysis'
data = pd.concat([pd.read_csv(op.join(outfolder,'selected_perc.csv'), index_col = 0),
                             pd.read_csv(op.join(outfolder,'selected_img.csv'),index_col=0)], ignore_index = True)
data['combined'] = data['kind'].str.cat(data[["subs"]].astype(str), sep="_")
data = data.groupby('kind').mean()
#specific = data[data['subs']==3]
specific = data.copy()
header = specific.index
#header = specific['combined'].values
specific = specific[[str(i) for i in range(10)]].transpose()
specific.columns = header
specific.corr()
#specific.corr(method =  pearsonr_pval)


kind,1,2,3,4,5,all,in,up
kind,,,,,,,,
1,1.000000,0.083683,0.116383,0.445641,-0.026559,0.552261,-0.195125,0.313028
2,0.083683,1.000000,0.302968,0.193406,-0.082884,0.660586,0.021401,0.219215
3,0.116383,0.302968,1.000000,0.419765,0.527223,0.534245,0.201323,0.105404
4,0.445641,0.193406,0.419765,1.000000,0.160614,0.125128,-0.027649,-0.222326
5,-0.026559,-0.082884,0.527223,0.160614,1.000000,0.275001,0.700152,0.254823
all,0.552261,0.660586,0.534245,0.125128,0.275001,1.000000,0.162011,0.609489
in,-0.195125,0.021401,0.201323,-0.027649,0.700152,0.162011,1.000000,0.209071
up,0.313028,0.219215,0.105404,-0.222326,0.254823,0.609489,0.209071,1.000000


In [8]:
import pandas as pd
%run general_tools.ipynb
import os.path as op

infolder, outfolder = find_folder()
outfolder = 'D:\\Ilya_study\\Analysis'
data = pd.concat([pd.read_csv(op.join(outfolder,'selected_perc.csv'), index_col = 0),
                             pd.read_csv(op.join(outfolder,'selected_img.csv'),index_col=0)], ignore_index = True)
data['combined'] = data['kind'].str.cat(data[["subs"]].astype(str), sep="_")
data = data.groupby('kind').mean()
data.head(n=20)

,0,1,2,3,4,5,6,7,8,9,subs
kind,,,,,,,,,,,
1,0.531863,0.683824,0.585784,0.610294,0.607843,0.649510,0.661765,0.642157,0.703431,0.666667,10
2,0.534314,0.502451,0.583333,0.507353,0.566176,0.539216,0.514706,0.531863,0.607843,0.561275,10
3,0.514706,0.534314,0.566176,0.512255,0.468137,0.512255,0.495098,0.534314,0.546569,0.512255,10
4,0.485294,0.502451,0.519608,0.500000,0.468137,0.522059,0.531863,0.477941,0.534314,0.492647,10
5,0.519608,0.517157,0.536765,0.497549,0.492647,0.507353,0.534314,0.551471,0.512255,0.504902,10
all,0.522059,0.568627,0.588235,0.529412,0.558824,0.534314,0.541667,0.602941,0.629902,0.598039,10
in,0.614750,0.568182,0.605615,0.487522,0.536765,0.515597,0.619875,0.596034,0.539216,0.595365,10
up,0.547906,0.585784,0.564171,0.548574,0.588681,0.517602,0.567068,0.588681,0.588681,0.554367,10


# Computing imagery coeficients' maps

In [5]:
%run general_tools.ipynb
%run EEG_auxiliary_module.ipynb
data=list()
labels = list()
trials=4
results=[]# Loading first S02 session
durs=[]
dictr = {"svm_imag_coefs.pkl":results, "svm_imag_duration.pkl":durs}
times = [(0.6,1.6),(1.6,2.6),(2.6,3.6),(3.6,4.6),(4.6,5.6)]
for time in times:
    result=[]
    for i,j in enumerate(subs):
        fname = op.join(outfolder,'S'+j+'_imag_aver-epo.fif')
        epochs = mne.read_epochs(fname)
        cv=len(epochs['31'])
        X, Y = convert_epochs_to_2D_array(epochs, times = time)
        confusion, duration = run_svm_coef(X, Y)
        result.append(confusion)
        durs.append(duration)
        update_progress(i/len(subs))
    results.append(result)
    save_all_pickle(dictr = dictr)
os.system('say "your program has finished"')

Progress: [###################-] 93.8%


TypeError: save_all_pickle() missing 1 required positional argument: 'folder'

# Computing perception coeficients' maps

In [ ]:
%run general_tools.ipynb
%run EEG_auxiliary_module.ipynb
data=list()
labels = list()
trials=4
results=[]
durs=[]
dictr = {"svm_perc_coefs.pkl":results, "svm_perc_duration.pkl":durs}
for time in prc_time:
    result=[]
    for i,j in enumerate(subs):
        fname = op.join(outfolder,'S'+j+'_perc_aver-epo.fif')
        epochs = mne.read_epochs(fname)
        cv=len(epochs['101'])
        X, Y = convert_epochs_to_2D_array(epochs, times = time)
        confusion, duration = run_svm_coef(X, Y)
        result.append(confusion)
        durs.append(duration)
        update_progress(i/len(subs))
    results.append(result)
    save_all_pickle(dictr = dictr)
os.system('say "your program has finished"')

In [ ]:
import pickle
results = pickle.load( open( "svm_prc_accuracy.pkl", "rb" ) )

In [ ]:
isolate_perc_5(df_perc)

In [ ]:
from scipy.spatial.distance import pdist, squareform
import numpy as np
confusion = results[14]
print(f'Accuracy of the upright stimuli is {np.mean(squareform(confusion[0:50,0:50]))*100:.4f}%')
print(f'Accuracy of the upright unfamiliary stimuli is {np.mean(squareform(confusion[0:25,0:25]))*100:.4f}%')
print(f'Accuracy of the upright famous stimuli is {np.mean(squareform(confusion[25:50,25:50]))*100:.4f}%')
print(f'Accuracy of the inverted stimuli is {np.mean(squareform(confusion[50:,50:]))*100:.4f}%')
print(f'Accuracy of the inverted unfamiliar stimuli is {np.mean(squareform(confusion[50:75,50:75]))*100:.4f}%')
print(f'Accuracy of the inverted famous stimuli is {np.mean(squareform(confusion[75:,75:]))*100:.4f}%')
print(f'Accuracy of the entire set is {np.mean(squareform(confusion))*100:.4f}%')

# Running face space analysis

### Computing permutations

In [52]:
import pickle
import numpy as np
from sklearn import manifold
from scipy.spatial import procrustes
%run EEG_auxiliary_module.ipynb
%run general_tools.ipynb

infolder, outfolder = find_folder()

data_prc = load_pkl(outfolder,'svm_prc_accuracy.pkl')
data_img = load_pkl(outfolder,'svm_img_accuracy.pkl')

data_prc.pop(-3)
data_prc = np.array(data_prc)
data_prc = np.mean(data_prc[:,25:50,25:50],axis = 0)
data_prc.shape #25 X 25

data_img = np.array(data_img)
data_img = np.delete(data_img,-3,1)
data_img = np.mean(data_img,axis=1)
data_img.shape # 6 X 5 X 5
space_img = [cmdscale(data_img[i,:,:])[0] for i in range(data_img.shape[0])]

out = []
for i in range(len(space_img)):
    update_progress(i/len(space_img))
    outt = []
    for j in range(10000):
        idx = np.random.permutation(data_prc.shape[0])[:5]
        temp = data_prc[np.ix_(idx,idx)]
        temp = cmdscale(temp)[0]
        _,_,d = procrustes(temp[:5,:4],space_img[i][:5,:4])
        outt.append(d)
    out.append(outt)
dictr = {"face_space_perms.pkl":out}
save_all_pickle(outfolder,dictr = dictr)

Progress: [#################---] 83.3%


In [16]:
import pickle
import numpy as np
from sklearn import manifold
from scipy.spatial import procrustes as procr
import pandas as pd
from scipy.spatial.distance import squareform, pdist
%run EEG_auxiliary_module.ipynb
%run general_tools.ipynb

infolder, outfolder = find_folder()

data_prc = load_pkl(outfolder,'svm_prc_accuracy.pkl')

data_prc.pop(-3)
data_prc = np.array(data_prc)
data_prc = np.mean(data_prc[:,25:50,25:50],axis = 0)
data_prc.shape #25 X 25

stims = pd.read_csv('C:\\Users\\danne\\Dropbox\\Ilya_study\\Stimuli\\stims.csv')
dist = pd.DataFrame(squareform(pdist(stims.T)),columns=stims.columns,index=stims.columns)
ims_names, prc_names, _ = import_names()
dist_imag = dist.loc[ims_names,ims_names]
dist_imag = cmdscale(dist_imag)[0]


out = []
for j in range(10000):
    idx = np.random.permutation(data_prc.shape[0])[:5]
    temp = data_prc[np.ix_(idx,idx)]
    temp = cmdscale(temp)[0]
    _,_,d = procr(temp[:5,:4],dist_imag[:5,:4])
    out.append(d)
dictr = {"TO_face_space_perms.pkl":out}
save_all_pickle(outfolder,dictr = dictr)

In [22]:
a = np.array(out)
d

{'rotation': array([[ 0.21446691,  0.34061697, -0.88088977,  0.24903262],
        [ 0.60014587, -0.34176884, -0.18377916, -0.69945995],
        [ 0.1287567 , -0.81824739, -0.1310518 ,  0.5447186 ],
        [ 0.7597701 ,  0.31248258,  0.41603349,  0.38989765]]),
 'scale': 0.00010095297014829635,
 'translation': array([-6.69327155e-17, -2.27411107e-17, -1.02584338e-17, -8.81229335e-18])}

In [24]:
from scipy.stats import percentileofscore
data_prc = load_pkl(outfolder,'svm_prc_accuracy.pkl')
data_prc = np.array(data_prc)
data_prc = np.mean(data_prc[:,:50,:50],axis = 0)
data_prc_5 = isolate_perc_5(data_prc)
_,_,d = procr(data_prc_5[:5,:4],dist_imag[:5,:4])
percentileofscore(a,d)

99.52

### Computing probabilities

In [53]:
from scipy.stats import percentileofscore
import numpy as np
from sklearn import manifold
from scipy.spatial import procrustes
%run general_tools.ipynb
%run EEG_auxiliary_module.ipynb

infolder, outfolder = find_folder()
data_prc = load_pkl(outfolder,'svm_prc_accuracy.pkl')
data_img = load_pkl(outfolder,'svm_img_accuracy.pkl')
out = load_pkl(outfolder,'face_space_perms.pkl')

data_prc.pop(-3)
data_prc = np.array(data_prc)
data_prc = np.mean(data_prc[:,:50,:50],axis = 0)
data_prc_5 = isolate_perc_5(data_prc)
print(data_prc_5)
data_prc_5 = cmdscale(data_prc_5)[0]
print(data_prc_5.shape)

data_img = np.array(data_img)
data_img = np.delete(data_img,-3,1)
data_img = np.mean(data_img,axis=1)
space_img = [cmdscale(data_img[i,:,:])[0] for i in range(data_img.shape[0])]
print(space_img[0].shape)
prcg = []
for i in range(6):
    _,_,d = procrustes(data_prc_5[:5,:4],space_img[i][:5,:4])
    prcg.append(percentileofscore(np.array(out[i]),d))
prcg


[[0.         0.55089962 0.59375    0.56557765 0.56202652]
 [0.55089962 0.         0.60724432 0.51609848 0.5764678 ]
 [0.59375    0.60724432 0.         0.59422348 0.59422348]
 [0.56557765 0.51609848 0.59422348 0.         0.55255682]
 [0.56202652 0.5764678  0.59422348 0.55255682 0.        ]]
(5, 4)
(5, 5)


[32.49, 21.32, 37.17, 46.52, 15.06, 3.17]

### Computing upright vs inverted

In [20]:
from scipy.stats import percentileofscore
import numpy as np
from sklearn import manifold
from scipy.spatial import procrustes
%run general_tools.ipynb
%run EEG_auxiliary_module.ipynb

def perm_mat(mat):
    idx = np.random.permutation(mat.shape[0])
    mat = mat[np.ix_(idx,idx)]
    mat = cmdscale(mat)
    return mat[0]

n_perms = 10000
infolder, outfolder = find_folder()
data_prc = load_pkl(outfolder,'svm_prc_accuracy.pkl')
out = load_pkl(outfolder,'face_space_perms.pkl')
data_prc.pop(-3)
data_prc = np.array(data_prc)
up_unf = np.mean(data_prc[:,:25,:25],axis = 0)
up_fam = np.mean(data_prc[:,25:50,25:50],axis = 0)
in_unf = np.mean(data_prc[:,50:75,50:75],axis = 0)
in_fam = np.mean(data_prc[:,75:,75:],axis = 0)

up_unf_space = cmdscale(up_unf)[0]
up_fam_space = cmdscale(up_fam)[0]
in_unf_space = cmdscale(in_unf)[0]
in_fam_space = cmdscale(in_fam)[0]

unf_d = []
fam_d = []
for i in range(n_perms):
    update_progress(i/n_perms)
    _,_,d = procrustes(up_unf_space[:25,:20],perm_mat(in_unf)[:25,:20])
    unf_d.append(d)
    _,_,d = procrustes(up_fam_space[:25,:20],perm_mat(in_fam)[:25,:20])
    fam_d.append(d)

_,_,d = procrustes(up_unf_space[:25,:20],in_unf_space[:25,:20])
print(percentileofscore(np.array(unf_d),d))
_,_,d = procrustes(up_fam_space[:25,:20],in_fam_space[:25,:20])
print(percentileofscore(np.array(fam_d),d))

Progress: [####################] 100.0%
0.51
5.77


In [3]:
data_prc.shape

(50, 50)

In [12]:
%run general_tools.ipynb
import numpy as np
from pathlib import Path, PurePath

infolder, outfolder = find_folder()
data = load_pkl(outfolder,'svm_img_accuracy.pkl')
data = data[5]
temp = [np.savetxt(PurePath(outfolder,"conf_imag_"+str(i+2).zfill(2)+'.csv'), 
            data[i], delimiter=",") for i in range(len(data))]


In [7]:
len(data)

17

In [1]:
%run EEG_auxiliary_module.ipynb
%run general_tools.ipynb
infolder, outfolder = find_folder()
data_prc = load_pkl(outfolder,'svm_prc_accuracy.pkl')

In [2]:
len(data_prc)

17